# Augmentation_6
Calculate std_dev from only the nn array, rather than all data

## Import libraries

In [1]:
#import libraries
import pandas as pd
import numpy as np
import random as rand

In [2]:
#import datasets
data = pd.read_csv("./dataset.csv")
pd.set_option('display.max_row',None)
pd.set_option('display.max_column',None)

## Separate Petting/Non-Petting

In [3]:
#separate the datasets Petting and the datasets that are not Petting
petting_d= data[data['Petting'] == 1]
non_petting_d= data[data['Petting'] == 0]

In [4]:
#Datasets preprocessing
def data_preprocessing(data):
    data.reset_index(inplace=True)
    data=data.drop(['index'],axis=1)
    data= data.drop(['Eating / Cooking', 'Moved It', 'Petting','TV / Radio','Talking'], axis=1)
    return data

## SMOTE

In [5]:
class SMOTE:
    def __init__(self,data,k,pad,synth_number,skew):
        self.data=data
        self.k=k
        self.pad=pad
        self.synth_n=synth_number
        self.skew = skew
        
    # Make the dataframe into list and put the index
    def making_list(self,data):
        data_list=data.values.tolist()
        a=0
        for x in data_list:
            x.insert(0,a)                                       #put the index of the data in the front
            a+=1
        return data_list

    # Calculate eucledian distance
    def eucledian(self,p1,p2):
        dist = np.sqrt(np.sum((p1-p2)**2))
        return dist
    
    # SMOTE algorithm
    def smote_al (self):
        data=self.making_list(self.data)
        feat_cnt = len(data[0])-1
        feat_start = 1                                          #start position of features in dataset, usually first column are IDs
        data_avg = [0 for i in range(feat_start+feat_cnt)] 
        data_sd = [0 for i in range(feat_start+feat_cnt)]
        data_med = [0 for i in range(feat_start+feat_cnt)]      #med = median
        data_skew = [0 for i in range(feat_start+feat_cnt)]
        k = self.k                                               #number of nearest neighbors (e.g. 3)
        pad = self.pad                                           #Padding - percentage of std dev to add to each feature (2 = 95th percentile)
        skew_x = self.skew                                       #Skew factor ... >1 increases skew, <1 decreases skew, =1 no effect
        synth_n = self.synth_n                                   #number of synthetic samples to make
        synth = [[] for i in range(synth_n)]                    #placeholder array for synthetic dataset
        nn= [[] for i in range(len(data))]
        
        #Find nearest neighbors
        dist_list=[]                              
        for i in data:                                      
            idx = i[0]                                          #get current sample index
            for m in range(len(data)):
                dist=self.eucledian(np.array(data[idx][1:]),np.array(data[m][1:])) #get the eucledian distance between the current data and other data
                a=(dist,idx,m)
                dist_list.append(list(a))
            dist_list.sort(key=lambda x:x[0])                  #sort the list by the distance
            for j in range(k):
                nn[idx].append(dist_list[j+1][2])               #append the data index
            dist_list=[]
        
        # Calc std devs 
        '''calculate the std dev for feature j across all rows of data, put in data_sd array'''
        for j in range(feat_start, feat_start+feat_cnt):
            temp_arr=[]
            for i in range(len(data)):
                temp_arr.append(data[i][j])                     #if dataset is numpy array, this could be done more cleanly w/o pulling data out to temp array
            data_avg[j] = np.mean(temp_arr)
            data_sd[j] = np.std(temp_arr)
            data_med[j] = np.median(temp_arr)
        
        #Calc skewness
        '''calculate skewness of dataset for each feature j'''
        for j in range(feat_start, feat_start+feat_cnt):
            if data_avg[j]-data_med[j]==0 :  # when avg, med, sd = 1.0, 1.0, 0.0
                data_skew[j] = 0
            else:
                data_skew[j] = (data_avg[j]-data_med[j])/ data_sd[j]
        
        # Create synthetic samples
        '''Randomly selecting samples, then use the difference between nearest neighbors and std dev to calcuate new feature values with perturbation'''
        for i in range(synth_n):    
            idx = rand.randint(0, len(data)-1)                      #get random sample index
            for j in range(feat_start, feat_start+feat_cnt):
                diff = 0                                            #variable for holding difference between nearest neighbor
                nn_sd = [0 for i in range(feat_start+feat_cnt)]
                gap = rand.uniform(-1.0,1.0)                       #random decimal number between -1 and 1
                temp_list = [] 
                for q in range(k):
                    temp = nn[idx][q]
                    temp_list.append(temp)
                    diff += abs(data[temp][j]) - abs(data[idx][j])    #sum absolute difference across all nearest neighbors
                nn_sd[j] = np.std(temp)
                diff = diff/k                                         #divide total by k to get avg diff
                diff = diff + (nn_sd[j]*pad)                         #add in the std_dev * padding  to difference to create perturbations
                gap2 = rand.uniform(0, 1.0)                          #random decimal number between -1 and 1
                if data_skew[j] == -np.inf:
                    data_skew[j] = -999
                skew_shift = (data_skew[j] * skew_x) - data_skew[j]     #calc skew_shift based on skew_x value
                
                if skew_shift == -np.inf:                             #To fix the error
                    skew_shift = -999
                elif skew_shift == np.inf:
                    skew_shift = 999
                elif np.isnan(skew_shift):
                    skew_shift = 0
                if data_avg[j] == -np.inf:
                    data_avg[j] = -999
                elif data_avg[j] == np.inf:
                    data_avg[j] = 999
                elif np.isnan(data_avg[j]):
                    data_avg[j] = 0
                if diff == -np.inf:
                    diff = -999
                elif diff == np.inf:
                    diff = 999
                elif np.isnan(diff):
                    diff = 0
                if gap == -np.inf:
                    gap = -999
                elif gap == np.inf:
                    gap = 999
                elif np.isnan(gap):
                    gap = 0
                if data[idx][j] == -np.inf:
                    data[idx][j] = -999
                elif data[idx][j] == np.inf:
                    data[idx][j] = 999
                elif np.isnan(data[idx][j]):
                    data[idx][j] = 0  
                skew_shift = data_avg[j] * skew_shift * gap2            #Apply skew_shift to average value, to get shift in unit_scale
                val = data[idx][j] + (diff*gap) + skew_shift
                if val == -np.inf:
                    val = -999
                elif val == np.inf:
                    val = 999
                elif np.isnan(val):
                    val = 0
                synth[i].append(val)
                
        #Make into Dataframe
        '''Append synth array onto the original dataset array'''
        len_data=len(self.data)
        for i in range(self.synth_n):
            self.data.loc[len_data+i]=synth[i]
        return self.data

In [6]:
# Merging petting and non_petting datasets into one dataframe
def into_one_dataframe(k, pad, synth_size, skew, t_1, t_0):
    p=SMOTE(t_1, k, pad, synth_size+5, skew)
    q=SMOTE(t_0, k, pad, synth_size, skew)
    petting_data=p.smote_al()
    non_petting_data=q.smote_al()
    petting_data['Petting'] = 1
    non_petting_data['Petting'] = 0
    merged_data=pd.concat([petting_data,non_petting_data]) 
    return merged_data

## Random Forest

In [7]:
# Import libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.metrics import roc_curve, roc_auc_score

In [8]:
def random_forest(data):
    # Split the data into features (X) and target (y)
    X = data.drop('Petting', axis=1)
    y = data['Petting']
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    #Random Forest
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    y_pred_prob = rf.predict_proba(X_test)[:, 1]
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    return accuracy, roc_auc

## Main Function

In [9]:
def main(k, pad, synth_size, skew, t_1, t_0):
    data_after_smote=into_one_dataframe(k, pad, synth_size, skew, t_1, t_0)
    rf_acc, rf_auc = random_forest(data_after_smote)
    return rf_acc, rf_auc

## Check ACC, AUC for each parameters

In [10]:
def check_acc_auc(k, pad, synth_size, skew, t_1, t_0):
    acc_Avg = 0
    auc_Avg = 0
    for i in range(5):
        acc, auc = main(k, pad, synth_size, skew, t_1.copy(), t_0.copy())        
        acc_Avg += acc
        auc_Avg += auc
    acc_Avg = acc_Avg/5
    auc_Avg = auc_Avg/5
    print('for k: ', k,' pad: ', pad,' synth_size: ', synth_size, 'for skew: ', skew)
    print('average_accuaracy : ', round(acc_Avg, 5))
    print('average_auc : ', round(auc_Avg, 5))
    return acc_Avg, auc_Avg

In [11]:
data_list=[]
petting=data_preprocessing(petting_d)
non_petting=data_preprocessing(non_petting_d)

In [12]:
# Run one by one 
# [k, pad, synth_size, skew]
params_list = [[2, 1, 40, 0.01], [2, 1, 40, 0.5], [2, 1, 40, 1.0], [2, 1, 40, 1.5], [2, 1, 40, 2.0],
              [2, 1, 80, 0.01], [2, 1, 80, 0.5], [2, 1, 80, 1.0], [2, 1, 80, 1.5], [2, 1, 80, 2.0],
              [2, 2, 40, 0.01], [2, 2, 40, 0.5], [2, 2, 40, 1.0], [2, 2, 40, 1.5], [2, 2, 40, 2.0],
              [2, 2, 80, 0.01], [2, 2, 80, 0.5], [2, 2, 80, 1.0], [2, 2, 80, 1.5], [2, 2, 80, 2.0],
              [3, 1, 40, 0.01], [3, 1, 40, 0.5], [3, 1, 40, 1.0], [3, 1, 40, 1.5], [3, 1, 40, 2.0],
              [3, 1, 80, 0.01], [3, 1, 80, 0.5], [3, 1, 80, 1.0], [3, 1, 80, 1.5], [3, 1, 80, 2.0],
              [3, 2, 40, 0.01], [3, 2, 40, 0.5], [3, 2, 40, 1.0], [3, 2, 40, 1.5], [3, 2, 40, 2.0],
              [3, 2, 80, 0.01], [3, 2, 80, 0.5], [3, 2, 80, 1.0], [3, 2, 80, 1.5], [3, 2, 80, 2.0],
              [4, 1, 40, 0.01], [4, 1, 40, 0.5], [4, 1, 40, 1.0], [4, 1, 40, 1.5], [4, 1, 40, 2.0],
              [4, 1, 80, 0.01], [4, 1, 80, 0.5], [4, 1, 80, 1.0], [4, 1, 80, 1.5], [4, 1, 80, 2.0],
              [4, 2, 40, 0.01], [4, 2, 40, 0.5], [4, 2, 40, 1.0], [4, 2, 40, 1.5], [4, 2, 40, 2.0],
              [4, 2, 80, 0.01], [4, 2, 80, 0.5], [4, 2, 80, 1.0], [4, 2, 80, 1.5], [4, 2, 80, 2.0]]

In [13]:
for param in params_list:
    temp_acc, temp_auc = check_acc_auc(param[0], param[1], param[2], param[3], petting, non_petting)
    data_list.append([param[0], param[1], param[2], param[3], temp_acc, temp_auc])

for k:  2  pad:  1  synth_size:  40 for skew:  0.01
average_accuaracy :  0.93
average_auc :  0.97318
for k:  2  pad:  1  synth_size:  40 for skew:  0.5
average_accuaracy :  0.91
average_auc :  0.92738
for k:  2  pad:  1  synth_size:  40 for skew:  1.0
average_accuaracy :  0.83
average_auc :  0.8941
for k:  2  pad:  1  synth_size:  40 for skew:  1.5
average_accuaracy :  0.94
average_auc :  0.98708
for k:  2  pad:  1  synth_size:  40 for skew:  2.0
average_accuaracy :  0.95
average_auc :  0.96938
for k:  2  pad:  1  synth_size:  80 for skew:  0.01
average_accuaracy :  0.96111
average_auc :  0.98465
for k:  2  pad:  1  synth_size:  80 for skew:  0.5
average_accuaracy :  0.92778
average_auc :  0.95302
for k:  2  pad:  1  synth_size:  80 for skew:  1.0
average_accuaracy :  0.92778
average_auc :  0.95578
for k:  2  pad:  1  synth_size:  80 for skew:  1.5
average_accuaracy :  0.95556
average_auc :  0.97188
for k:  2  pad:  1  synth_size:  80 for skew:  2.0
average_accuaracy :  0.95
average_au

In [14]:
acc_auc_summary_df = pd.DataFrame(data=data_list, 
                                  columns = ['k', 'pad', 'synth_size', 'skewed', 'average_accuracy', 'average_auc'])
acc_auc_summary_df

,k,pad,synth_size,skewed,average_accuracy,average_auc
0,2,1,40,0.01,0.930000,0.973179
1,2,1,40,0.50,0.910000,0.927375
2,2,1,40,1.00,0.830000,0.894096
3,2,1,40,1.50,0.940000,0.987077
4,2,1,40,2.00,0.950000,0.969381
5,2,1,80,0.01,0.961111,0.984650
6,2,1,80,0.50,0.927778,0.953018
7,2,1,80,1.00,0.927778,0.955777
8,2,1,80,1.50,0.955556,0.971884
9,2,1,80,2.00,0.950000,0.982470


In [15]:
acc_auc_summary_df.to_csv('./augmentation_6.csv')